In [1]:
import numpy as np
import util

In [2]:
n=15
d=15
M = np.random.rand(n,d)
l,v,r = np.linalg.svd(M)
v_full = np.append(np.diag(v), np.zeros((n-d,d)), axis=0)
RM = np.dot(np.dot(l,v_full), r)
print(np.linalg.norm(M - RM, ord='fro'))

2.04856497309e-14


In [3]:
# rank 3 approx
rank = 3
v3 = [v[i] if i < rank else 0 for i in range(len(v))]
v3_full = np.append(np.diag(v3), np.zeros((n-d,d)), axis=0)
RM3 = np.dot(np.dot(l,v3_full), r)
print(np.linalg.norm(M - RM3, ord='fro'))

2.92706191089


In [4]:
# set half of the values to 0
mask = np.random.randint(0,2,size=M.shape).astype(np.bool)
z = np.zeros((n,d))
RM3_zero = RM3.copy()
RM3_zero[mask] = z[mask]

In [5]:
def calc_error(R, P, Q, rank, beta):
    e = 0
    for i in xrange(len(R)):
        for j in xrange(len(R[i])):
            if R[i][j] > 0:
                e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                for k in xrange(rank):
                    e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
    return e

In [6]:
def matrix_factorization_sgd(R, P, Q, K, steps=5000, alpha=0.0001, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR =  np.dot(P,Q)
    return P, Q.T

In [7]:
def matrix_factorization_SVRG(R, P, Q, rank, K, T, alpha, beta):
    Q = Q.T
    P_last = P
    Q_last = Q
    errors = [calc_error(R, P, Q, rank, beta)]
    #print(errors)
    e_old = 1000000
    for K_i in range(K):
        P_tilde = P_last
        Q_tilde = Q_last
        P_mu_tilde = np.zeros(P.shape)
        Q_mu_tilde = np.zeros(Q.shape)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(rank):
                        P_mu_tilde[i][k] = P_mu_tilde[i][k]  + 2 * eij * Q_tilde[k][j] - beta * P_tilde[i][k]
                        Q_mu_tilde[k][j] = Q_mu_tilde[k][j] + 2 * eij * P_tilde[i][k] - beta * Q_tilde[k][j]
        P_mu_tilde = P_mu_tilde/rank
        Q_mu_tilde = Q_mu_tilde/rank
        for _ in range(T):
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j] > 0:
                        eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                        for k in range(rank):
                            P[i][k] = P[i][k] + alpha * ((2 * eij * Q[k][j] - beta * P[i][k]) -(2 * eij * Q_tilde[k][j] - beta * P_tilde[i][k]) + P_mu_tilde[i][k])
                            Q[k][j] = Q[k][j] + alpha * ((2 * eij * P[i][k] - beta * Q[k][j]) -
                             (2* eij * P_tilde[i][k] - beta * Q_tilde[k][j]) + Q_mu_tilde[k][j])
        P_last = P
        Q_last = Q
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(rank):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        #print(errors)
        errors.append(e)
        #print(errors)
        if e < 0.001 or e > e_old:
            break
        e_old = e
    return P, Q.T, errors

In [8]:
def quantize_num(n, s, qtype, vmin, vmax):
    n = n / s
    if n > vmax:
        return qtype(vmax)
    elif n < vmin:
        return qtype(vmin)
    else:
        n2 = qtype(n)
        if np.random.rand() > n%1 and n2 < vmax:
            return n2+1
        return n2

In [9]:
def matrix_factorization_SVRG_LP(R, P, Q, rank, K, T, alpha, beta, bits, data_type):
    Q = Q.T
    P_last = P
    Q_last = Q
    max_val = 1.0*((2**(bits-1))-1); min_val = -1.0*((2**(bits-1)))
    s = 1/(1.0*2**(bits-1))
    errors = [calc_error(R,P.astype(float)*s, Q.astype(float)*s, rank, beta)]
    e_old = 1000000
    for K_i in range(K):
        P_tilde = P_last
        Q_tilde = Q_last
        P_mu_tilde = np.zeros(P.shape)
        Q_mu_tilde = np.zeros(Q.shape)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:].astype(float)*s,Q[:,j].astype(float)*s)
                    for k in range(rank):
                        P_mu_tilde[i][k] = P_mu_tilde[i][k]  + 2 * eij * Q_tilde[k][j].astype(float)*s - beta * P_tilde[i][k].astype(float)*s
                        Q_mu_tilde[k][j] = Q_mu_tilde[k][j] + 2 * eij * P_tilde[i][k].astype(float)*s - beta * Q_tilde[k][j].astype(float)*s
        P_mu_tilde = P_mu_tilde/rank
        Q_mu_tilde = Q_mu_tilde/rank
        for _ in range(T):
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j] > 0:
                        eij = R[i][j] - np.dot(P[i,:].astype(float)*s,Q[:,j].astype(float)*s)
                        for k in range(rank):
                            pik = float(P[i][k])*s; qkj = float(Q[k][j])*s;
                            qtkj = float(Q_tilde[k][j])*s; ptik = float(P_tilde[i][k])*s;
                            qkj = float(Q[k][j])*s; qtkj = float(Q_tilde[k][j])*s;
                            pik = pik + alpha * ((2 * eij * qkj - beta * pik) - (2 * eij * qtkj - beta * ptik) + P_mu_tilde[i][k])
                            qkj = qkj + alpha * ((2 * eij * pik - beta * qkj) - (2* eij * ptik - beta * qtkj) + Q_mu_tilde[k][j])
                            P[i][k] = quantize_num(pik, s, data_type, min_val, max_val)
                            Q[k][j] = quantize_num(qkj, s, data_type, min_val, max_val)
        P_last = P
        Q_last = Q
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:].astype(float)*s,Q[:,j].astype(float)*s), 2)
                    for k in xrange(rank):
                        e = e + (beta/2) * ( pow(P[i][k].astype(float)*s,2) + pow(Q[k][j].astype(float)*s,2) )
        errors.append(e)
        if e < 0.001 or e > e_old:
            break
        e_old = e
    return P.astype(float)*s, Q.T.astype(float)*s, errors

In [10]:
def matrix_factorization_HALP(R, P, Q, rank, K, T, alpha, beta, bits, data_type):
    Q = Q.T
    max_val = 1.0*((2**(bits-1))-1); min_val = -1.0*((2**(bits-1)))
    s = 0
    errors = [calc_error(R,P, Q, rank, beta)]
    e_old = 1000000
    ZP = np.zeros(P.shape,dtype=data_type)
    ZQ = np.zeros(Q.shape,dtype=data_type)
    s = 0.0
    step = 1
    for K_i in range(K):
        P = P + ZP.astype(float)*s
        Q = Q + ZQ.astype(float)*s
        P_mu_tilde = np.zeros(P.shape)
        Q_mu_tilde = np.zeros(Q.shape)
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:].astype(float)*s,Q[:,j].astype(float)*s)
                    for k in range(rank):
                        P_mu_tilde[i][k] = P_mu_tilde[i][k]  + 2 * eij * Q[k][j] - beta * P[i][k]
                        Q_mu_tilde[k][j] = Q_mu_tilde[k][j] + 2 * eij * P[i][k] - beta * Q[k][j]
        P_mu_tilde = P_mu_tilde/rank
        Q_mu_tilde = Q_mu_tilde/rank
        step+=0
        s = (0.05 /(2**(bits-1)))/step
        ZP = np.zeros(P.shape,dtype=data_type)
        ZQ = np.zeros(Q.shape,dtype=data_type)
        for _ in range(T):
            for i in range(len(R)):
                for j in range(len(R[i])):
                    if R[i][j] > 0:
                        eij = R[i][j] - np.dot(P[i,:]+ZP[i,:].astype(float)*s,Q[:,j]+ZQ[:,j].astype(float)*s)
                        for k in range(rank):
                            zpik = float(ZP[i][k])*s; zqkj = float(ZQ[k][j])*s
                            zpik = zpik - alpha * ((2 * eij * (Q[k][j]+zqkj) - beta * (P[i][k])+zpik) -(2 * eij * Q[k][j] - beta * P[i][k]) + P_mu_tilde[i][k])
                            zqkj = zqkj - alpha * ((2 * eij * (P[i][k]+zpik) - beta * (Q[k][j]+zqkj)) - (2* eij * P[i][k] - beta * Q[k][j]) + Q_mu_tilde[k][j])
                            ZP[i][k] = quantize_num(zpik, s, data_type, min_val, max_val)
                            ZQ[k][j] = quantize_num(zqkj, s, data_type, min_val, max_val)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:]+ZP[i,:].astype(float)*s,Q[:,j]+ZQ[:,j].astype(float)*s), 2)
                    for k in xrange(rank):
                        e = e + (beta/2) * ( pow(P[i][k]+float(ZP[i][k])*s,2) + pow(Q[k][j]+float(ZQ[k][j])*s,2) )
        errors.append(e)
        if e < 0.001 or e > e_old:
            break
        e_old = e
    return P + ZP.astype(float)*s, Q.T + ZQ.T.astype(float)*s, errors

In [11]:
def handle_SVRG(epochs, sgd_updates, P_init, Q_init, alpha, beta):
    rank = 3
    nP, nQ, errors = matrix_factorization_SVRG(RM3_zero, P_init, Q_init, rank, epochs, sgd_updates, alpha, beta)
    RM3_est = np.dot(nP,nQ.T)
    RM3_est_z = RM3_est.copy()
    RM3_est_z[mask] = z[mask]
    print(np.linalg.norm(RM3_est_z - RM3_zero, ord='fro'))
    print(np.linalg.norm(RM3_est - RM3, ord='fro'))
    return errors

In [12]:
def handle_SVRG_LP(epochs, sgd_updates, P_init, Q_init, bits, data_type, alpha, beta):
    rank = 3
    nP, nQ, errors = matrix_factorization_SVRG_LP(RM3_zero, P_init, Q_init, rank, epochs, sgd_updates, alpha, beta, bits, data_type)
    RM3_est = np.dot(nP,nQ.T)
    RM3_est_z = RM3_est.copy()
    RM3_est_z[mask] = z[mask]
    print(np.linalg.norm(RM3_est_z - RM3_zero, ord='fro'))
    print(np.linalg.norm(RM3_est - RM3, ord='fro'))
    return errors

In [13]:
def handle_HALP(epochs, sgd_updates, P_init, Q_init, bits, data_type, alpha, beta):
    rank = 3
    nP, nQ, errors = matrix_factorization_HALP(RM3_zero, P_init, Q_init, rank, epochs, sgd_updates, alpha, beta, bits, data_type)
    RM3_est = np.dot(nP,nQ.T)
    RM3_est_z = RM3_est.copy()
    RM3_est_z[mask] = z[mask]
    print(np.linalg.norm(RM3_est_z - RM3_zero, ord='fro'))
    print(np.linalg.norm(RM3_est - RM3, ord='fro'))
    return errors

In [14]:
P_gen = np.random.randint(2**15, size=(n, rank),dtype=np.int16)
Q_gen = np.random.randint(2**15, size=(d, rank),dtype=np.int16)
s = 1/(1.0*2**15)
P_full_gen = P_gen.astype(float)*s
Q_full_gen = Q_gen.astype(float)*s
alpha = 0.00001
beta = 0.02

In [15]:
errors_SVRG = handle_SVRG(5,1000,P_full_gen.copy(), Q_full_gen.copy(),alpha, beta)

2.09879632788
3.4247550068


In [ ]:
errors_LP_SVRG = handle_SVRG_LP(5,1000, P_gen.copy(), Q_gen.copy(), 16, np.int16, alpha, beta)

3.0970255392
4.37001816485


In [ ]:
errors_HALP = handle_HALP(5,1000, P_full_gen.copy(), Q_full_gen.copy(), 16, np.int16, alpha, beta)

In [ ]:
util.draw_line_graph2(errors_SVRG, errors_LP_SVRG, "error", "epochs")

In [ ]:
util.draw_line_graph3(errors_SVRG, errors_LP_SVRG, errors_HALP, "error", "epochs")